# Data Science Case Study

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/liddingtonb/Library/Caches/pypoetry/virtualenvs/case-study-E4SsEFgE-py3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 3699, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/nd/bjvm3cd953z10kb2c4z1rs4rnnf7fh/T/ipykernel_57068/727953073.py", line 1, in <module>
    import pandas as pd
ModuleNotFoundError: No module named 'pandas'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/liddingtonb/Library/Caches/pypoetry/virtualenvs/case-study-E4SsEFgE-py3.13/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 2194, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=tb_offset
    )
  File "/Users/liddingtonb/Library/Caches/pypoetry/virtualenvs/case-study-E4SsEFgE-py3.13/lib/python3.13/site-pa